**Введите ваше ФИО**

In [ ]:
Шинелёв Максим Денисович

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение объектов, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [1]:
import requests
from datetime import datetime
from PIL import Image
from urllib.request import urlopen
from IPython.display import display
import io


NASA_API_KEY = 'f6vOA6qgTllkGqJSBIrwXwCN2oPdNjmU26H3WKVd'
NASA_APOD_URL = 'https://api.nasa.gov/planetary/apod'
NASA_MARS_ROVER_URL = 'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos'
NASA_NEOS_URL = 'https://api.nasa.gov/neo/rest/v1/feed'
NASA_SPACE_WEATHER_URL = 'https://api.nasa.gov/DONKI/GST'


def get_apod():
    params = {'api_key': NASA_API_KEY}
    response = requests.get(NASA_APOD_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['title'], data['explanation'], data['url']
    else:
        return None, "Ошибка получения данных APOD", None


def get_mars_photos(rover_date, camera=None):
    params = {'earth_date': rover_date, 'api_key': NASA_API_KEY}
    if camera:
        params['camera'] = camera
    response = requests.get(NASA_MARS_ROVER_URL, params=params)
    if response.status_code == 200:
        return response.json().get('photos', []), []
    else:
        return [], "Ошибка получения данных с марсохода"


def get_neo_data(date):
    params = {'start_date': date, 'end_date': date, 'api_key': NASA_API_KEY}
    response = requests.get(NASA_NEOS_URL, params=params)
    if response.status_code == 200:
        return response.json()['near_earth_objects'], []
    else:
        return [], "Ошибка получения данных о ОСЗ"


def get_space_weather():
    params = {'api_key': NASA_API_KEY}
    response = requests.get(NASA_SPACE_WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json(), []
    else:
        return None, "Ошибка получения данных о космической погоде"


def main():
    while True:
        print("Выберите функцию:")
        print("1. Астрономическая картинка дня")
        print("2. Фотографии с марсохода")
        print("3. Объекты, сближающиеся с Землей")
        print("4. Данные о космической погоде")
        print("5. Выход\n")

        choice = input("Введите номер функции: ")

        if choice == '1':
            title, explanation, url = get_apod()
            if title:
                print(f"**{title}**")
                print(f"Описание: {explanation}")
                img = Image.open(io.BytesIO(requests.get(url).content))
                display(img)
            else:
                print(explanation)
            print()

        elif choice == '2':
            date = input("Введите дату (YYYY-MM-DD): ")
            try:
                datetime.strptime(date, '%Y-%m-%d')
                camera = input("Введите тип камеры (или оставьте пустым для всех): ")
                photos, error = get_mars_photos(date, camera)
                if error:
                  print(error)
                for photo in photos:
                    img = Image.open(io.BytesIO(requests.get(photo['img_src']).content))
                    display(img)
                print()
            except ValueError:
                print("Неверный формат даты.\n")

        elif choice == '3':
            date = input("Введите дату (YYYY-MM-DD): ")
            try:
                datetime.strptime(date, '%Y-%m-%d')
                neo_data, error = get_neo_data(date)
                if error:
                  print(error)
                for neo in neo_data[date]:
                  print(f"Название: {neo['name']}, размер: {neo['estimated_diameter']['meters']['estimated_diameter_max']} метров, потенциальная опасность: {neo['is_potentially_hazardous_asteroid']}")
                print()
            except ValueError:
              print("Неверный формат даты.\n")

        elif choice == '4':
            space_weather, error = get_space_weather()
            if error:
                print(error)
            else:
                for event in space_weather:
                    print(f"Время события: {event['startTime']}, индекс активности: {event['allKpIndex'][0]['kpIndex']}")
            print()

        elif choice == '5':
            print("Выход из приложения.")
            break

        else:
            print("Некорректный выбор. Пожалуйста, попробуйте снова.\n")


if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [6]:
import requests


API_URL = "https://api.artic.edu/api/v1/artworks"


def fetch_artworks(page=1):
    params = {
        'page': page,
        'limit': 10
    }
    response = requests.get(API_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return data['data'], data['pagination']
    else:
        print("Ошибка извлечения данных:", response.status_code)
        return [], {}


def filter_artworks_by_artist(artworks, artist_name):
    filtered_artworks = [
        artwork for artwork in artworks if artist_name.lower()
        in artwork.get('artist_display', '').lower()
    ]
    return filtered_artworks


def display_artwork_details(artwork):
    print(f"\nНазвание: {artwork.get('title', 'Неизвестно')}")
    print(f"Исполнитель: {artwork.get('artist_display', 'Неизвестно')}")
    print(f"Дата: {artwork.get('date_display', 'Неизвестно')}")
    print(f"Носитель: {artwork.get('medium_display', 'Неизвестно')}")
    print(f"Изображение: {artwork.get('image_url', 'Нет изображения')}\n")


def main():
    page = 1
    while True:
        artworks, pagination = fetch_artworks(page)
        if not artworks:
            print("Нет доступных произведений искусства.")
            break

        print(f"\nСтраница {pagination['current_page']} из {pagination['total_pages']}:")
        for index, artwork in enumerate(artworks, start=1):
            print(f"{index}. {artwork.get('title', 'Неизвестно')} (ID: {artwork['id']})")

        print("\nВыберите действие:")
        print("1. Показать подробности о произведении искусства")
        print("2. Перейти на следующую страницу")
        print("3. Перейти на предыдущую страницу")
        print("4. Фильтровать по имени художника")
        print("5. Выход")

        choice = input("Введите номер действия: ").strip().lower()

        if choice == '1':
            index = int(input("Введите номер произведения для просмотра деталей: ")) - 1
            if 0 <= index < len(artworks):
                display_artwork_details(artworks[index])
            else:
                print("Некорректный номер.")

        elif choice == '2':
            if pagination['next_url']:
                page += 1
            else:
                print("Вы на последней странице.")

        elif choice == '3':
            if pagination['prev_url']:
                page -= 1
            else:
                print("Вы на первой странице.")

        elif choice == '4':
            artist_name = input("Введите имя художника: ")
            filtered_artworks = filter_artworks_by_artist(artworks, artist_name)
            print(f"\nНайденные произведения искусства художника '{artist_name}':")
            for artwork in filtered_artworks:
                print(f"- {artwork.get('title', 'Неизвестно')} (ID: {artwork['id']})")

        elif choice == '5':
            print("Выход из программы.")
            break

        else:
            print("Некорректный выбор. Попробуйте еще раз.")


if __name__ == "__main__":
    main()


Страница 1 из 12632:
1. Sound of Silence (ID: 240967)
2. Sound of Silence (ID: 240956)
3. Wanted Poster Series #12a (ID: 240945)
4. Wanted Poster Series #11 (positive image) (ID: 240942)
5. Wanted Poster Series #14a (ID: 240933)
6. Sound of Silence (ID: 240953)
7. Wanted Poster Series #12 (ID: 240944)
8. Cabeza (or Youth) (ID: 229152)
9. Gideon (ID: 241448)
10. I Have a Dream (ID: 240936)

Выберите действие:
1. Показать подробности о произведении искусства
2. Перейти на следующую страницу
3. Перейти на предыдущую страницу
4. Фильтровать по имени художника
5. Выход
Введите номер действия: 2

Страница 2 из 12632:
1. Untitled (ID: 105282)
2. Grand Avenue, Chicago (ID: 63279)
3. Mrs. George Swinton (Elizabeth Ebsworth) (ID: 4749)
4. Gideon (ID: 240948)
5. Harvest Talk (ID: 111810)
6. Wanted Poster Series #14 (ID: 240932)
7. This, My Brother (ID: 152057)
8. Portrait of a Woman (ID: 117417)
9. Sound of Silence (ID: 240967)
10. Sound of Silence (ID: 240957)

Выберите действие:
1. Показать по

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [17]:
import requests
import json
from datetime import datetime, timedelta


COINGECKO_API_URL = 'https://api.coingecko.com/api/v3'


class CryptoPortfolio:
    def __init__(self):
        self.portfolio = {}


    def add_crypto(self, symbol, amount):
        if symbol in self.portfolio:
            self.portfolio[symbol] += amount
        else:
            self.portfolio[symbol] = amount


    def remove_crypto(self, symbol, amount):
        if symbol in self.portfolio:
            self.portfolio[symbol] -= amount
            if self.portfolio[symbol] <= 0:
                del self.portfolio[symbol]


    def get_portfolio_value(self, currency='usd'):
        total_value = 0
        for symbol, amount in self.portfolio.items():
            price = self.get_current_price(symbol, currency)[0]
            if price:
                total_value += (amount * price)
        return total_value


    def get_current_price(self, symbol, currency='usd'):
        response = requests.get(f"{COINGECKO_API_URL}/simple/price",
                                params={'ids': symbol,
                                        'vs_currencies': currency,
                                        'include_24hr_change': 'true'})
        if response.status_code == 200:
            data = response.json()
            return data[symbol][currency], data[symbol][f'{currency}_24h_change']
        else:
            return None, None


    def get_historical_data(self, symbol):
        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)
        response = requests.get(f"{COINGECKO_API_URL}/coins/{symbol}/market_chart/range", params={
            'vs_currency': 'usd',
            'from': start_date.timestamp(),
            'to': end_date.timestamp()
        })
        if response.status_code == 200:
            prices = json.loads(response.text)["prices"]
            return [(datetime.utcfromtimestamp(price[0] / 1000),
                     price[1]) for price in prices]
        else:
            return None


def main():
    portfolio = CryptoPortfolio()

    while True:
        print("\nВыберите действие:")
        print("1. Добавить криптовалюту в портфель")
        print("2. Удалить криптовалюту из портфеля")
        print("3. Посмотреть стоимость портфеля")
        print("4. Проверить изменения цен")
        print("5. Получить исторические данные")
        print("6. Выход\n")

        choice = input("Введите номер действия: ")

        if choice == '1':
            symbol = input("Введите символ криптовалюты: ").lower()
            amount = float(input("Введите количество: "))
            portfolio.add_crypto(symbol, amount)
            print(f"Криптовалюта {symbol} добавлена в портфель.")

        elif choice == '2':
            symbol = input("Введите символ криптовалюты: ").lower()
            amount = float(input("Введите количество для удаления: "))
            portfolio.remove_crypto(symbol, amount)
            print(f"Криптовалюта {symbol} обновлена в портфеле.")

        elif choice == '3':
            total_value = portfolio.get_portfolio_value()
            print(f"Общая стоимость портфеля: {total_value:.2f} USD")

        elif choice == '4':
            for symbol in portfolio.portfolio:
                price, change = portfolio.get_current_price(symbol)
                amount = portfolio.portfolio[symbol]
                if price:
                    print(f"{symbol.upper()}: Цена = {price:.2f} USD, Изменение за 24 часа = {change:.2f}%")

        elif choice == '5':
            symbol = input("Введите символ криптовалюты для получения исторических данных: ").lower()
            historical_data = portfolio.get_historical_data(symbol)
            if historical_data:
                print(f"Исторические данные для {symbol.upper()}:")
                for date, price in historical_data:
                    print(f"{date.strftime('%Y-%m-%d')} - {price:.2f} USD")
            else:
                print("Ошибка получения исторических данных.")

        elif choice == '6':
            print("Выход из приложения.")
            break

        else:
            print("Некорректный выбор. Пожалуйста, попробуйте снова.")


if __name__ == "__main__":
    main()


Выберите действие:
1. Добавить криптовалюту в портфель
2. Удалить криптовалюту из портфеля
3. Посмотреть стоимость портфеля
4. Проверить изменения цен
5. Получить исторические данные
6. Выход

Введите номер действия: 1
Введите символ криптовалюты: ethereum
Введите количество: 1
Криптовалюта ethereum добавлена в портфель.

Выберите действие:
1. Добавить криптовалюту в портфель
2. Удалить криптовалюту из портфеля
3. Посмотреть стоимость портфеля
4. Проверить изменения цен
5. Получить исторические данные
6. Выход

Введите номер действия: 4
ETHEREUM: Цена = 2433.47 USD, Изменение за 24 часа = -1.30%

Выберите действие:
1. Добавить криптовалюту в портфель
2. Удалить криптовалюту из портфеля
3. Посмотреть стоимость портфеля
4. Проверить изменения цен
5. Получить исторические данные
6. Выход

Введите номер действия: 1
Введите символ криптовалюты: bitcoin
Введите количество: 0.01
Криптовалюта bitcoin добавлена в портфель.

Выберите действие:
1. Добавить криптовалюту в портфель
2. Удалить крип

# Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код